### define

In [1]:
%autoreload 2
import torch
from torchvision import datasets, transforms
import argparse
from simple_conv_net_func import diff_mse
from simple_conv_net_func import conv2d_scalar, pool2d_scalar, relu_scalar, reshape_scalar, fc_layer_scalar
from simple_conv_net_func import conv2d_vector, pool2d_vector, relu_vector, reshape_vector, fc_layer_vector, im2col

In [2]:
N_batch = 4  # bacth size, default 64
C = 1  # number of input channels
S = 28  # input image size
B = 20  # number of filters
K = 5  # kernel size
fc1_out = 500  # fc1 layer output units
N_classes = 10
epochs_num = 20
alpha = 0.01
momentum = 0.5

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [4]:
x = torch.randn(N_batch, C, S, S).to(device)
# w = torch.randn(B, C, K, K).to(device)
# b = torch.randn(B).to(device)
x.shape#, w.shape, b.shape

torch.Size([4, 1, 28, 28])

In [5]:
x.get_device()

-1

## scalar

### conv

In [6]:
cl = torch.nn.Conv2d(in_channels=C, out_channels=B, kernel_size=K)
w_conv = cl.weight
b_conv = cl.bias
w_conv.shape, b_conv.shape

(torch.Size([20, 1, 5, 5]), torch.Size([20]))

In [7]:
with torch.no_grad():
    %time z_conv = conv2d_scalar(x, w_conv, b_conv, device)
z_conv.shape

CPU times: user 23.2 s, sys: 4.81 ms, total: 23.2 s
Wall time: 23.2 s


torch.Size([4, 20, 24, 24])

In [8]:
with torch.no_grad():
    %time torch_z_conv = cl(x)
torch_z_conv.shape

CPU times: user 784 µs, sys: 36 µs, total: 820 µs
Wall time: 599 µs


torch.Size([4, 20, 24, 24])

In [9]:
diff_mse(z_conv, torch_z_conv)

3.95232052990903e-15

### pool

In [10]:
with torch.no_grad():
    %time z_pool = pool2d_scalar(torch_z_conv, device)
z_pool.shape

CPU times: user 354 ms, sys: 4.11 ms, total: 359 ms
Wall time: 358 ms


torch.Size([4, 20, 12, 12])

In [11]:
with torch.no_grad():
    %time torch_z_pool = torch.max_pool2d(torch_z_conv, 2)
torch_z_pool.shape

CPU times: user 192 µs, sys: 10 µs, total: 202 µs
Wall time: 204 µs


torch.Size([4, 20, 12, 12])

In [12]:
diff_mse(z_pool, torch_z_pool)

0.0

### reshape

In [13]:
with torch.no_grad():
    %time z_reshaped = reshape_scalar(z_pool, device)
z_reshaped.shape

CPU times: user 90.4 ms, sys: 0 ns, total: 90.4 ms
Wall time: 89.9 ms


torch.Size([4, 2880])

In [14]:
with torch.no_grad():
    # %time torch_z_reshaped = torch.reshape(torch_z_pool, (N_batch, -1))
    %time torch_z_reshaped = torch_z_pool.view(N_batch, -1)
torch_z_reshaped.shape

CPU times: user 28 µs, sys: 1e+03 ns, total: 29 µs
Wall time: 31.2 µs


torch.Size([4, 2880])

In [15]:
diff_mse(z_reshaped, torch_z_reshaped)

0.0

### relu

In [16]:
with torch.no_grad():
    %time z_relu = relu_scalar(torch_z_reshaped, device)
z_relu.shape

CPU times: user 121 ms, sys: 58 µs, total: 121 ms
Wall time: 121 ms


torch.Size([4, 2880])

In [17]:
with torch.no_grad():
    %time torch_z_relu = torch.relu(torch_z_reshaped)
torch_z_relu.shape

CPU times: user 69 µs, sys: 4 µs, total: 73 µs
Wall time: 89.2 µs


torch.Size([4, 2880])

In [18]:
diff_mse(z_relu, torch_z_relu)

0.0

### fc

In [19]:
fc = torch.nn.Linear(torch_z_reshaped.size(-1), N_classes)
w_fc = fc.weight
b_fc = fc.bias

In [20]:
with torch.no_grad():
    %time z_fc = fc_layer_scalar(torch_z_reshaped, w_fc, b_fc, device)
z_fc.shape

CPU times: user 1.56 s, sys: 0 ns, total: 1.56 s
Wall time: 1.56 s


torch.Size([4, 10])

In [21]:
with torch.no_grad():
    %time torch_z_fc = fc(z_reshaped)
torch_z_fc.shape

CPU times: user 686 µs, sys: 42 µs, total: 728 µs
Wall time: 334 µs


torch.Size([4, 10])

In [22]:
diff_mse(z_fc, torch_z_fc)

1.6700226271315621e-13

## vector

#### im2col

In [23]:
with torch.no_grad():
    %time col = im2col(x[0], 5, device).t().contiguous()
col.shape

CPU times: user 7.91 ms, sys: 0 ns, total: 7.91 ms
Wall time: 7.53 ms


torch.Size([25, 576])

In [24]:
with torch.no_grad():
    %time torch_col = torch.functional.F.unfold(x, 5)[0]
torch_col.shape

CPU times: user 181 µs, sys: 11 µs, total: 192 µs
Wall time: 195 µs


torch.Size([25, 576])

In [25]:
diff_mse(col, torch_col)

0.0

### conv

In [26]:
cl = torch.nn.Conv2d(in_channels=C, out_channels=B, kernel_size=K)
w_conv = cl.weight
b_conv = cl.bias

In [27]:
with torch.no_grad():
    %time z_conv = conv2d_vector(x, w_conv, b_conv, device)
z_conv.shape

CPU times: user 96.2 ms, sys: 0 ns, total: 96.2 ms
Wall time: 29.7 ms


torch.Size([4, 20, 24, 24])

In [28]:
with torch.no_grad():
    %time torch_z_conv = cl(x)
torch_z_conv.shape

CPU times: user 570 µs, sys: 39 µs, total: 609 µs
Wall time: 389 µs


torch.Size([4, 20, 24, 24])

In [29]:
diff_mse(z_conv, torch_z_conv)

3.57074997486286e-15

### pool

In [30]:
with torch.no_grad():
    %time z_pool = pool2d_vector(torch_z_conv, device)
z_pool.shape

CPU times: user 141 ms, sys: 46 µs, total: 141 ms
Wall time: 140 ms


torch.Size([4, 20, 12, 12])

In [31]:
with torch.no_grad():
    %time torch_z_pool = torch.max_pool2d(torch_z_conv, 2)
torch_z_pool.shape

CPU times: user 155 µs, sys: 11 µs, total: 166 µs
Wall time: 168 µs


torch.Size([4, 20, 12, 12])

In [32]:
diff_mse(z_pool, torch_z_pool)

0.0

### reshape

In [33]:
with torch.no_grad():
    %time z_reshaped = reshape_vector(z_pool, device)
z_reshaped.shape

CPU times: user 35 µs, sys: 3 µs, total: 38 µs
Wall time: 40.1 µs


torch.Size([4, 2880])

In [34]:
with torch.no_grad():
#     %time torch_z_reshaped = torch.reshape(torch_z_pool, (N_batch, -1))
    %time torch_z_reshaped = torch_z_pool.view(N_batch, -1)
torch_z_reshaped.shape

CPU times: user 29 µs, sys: 2 µs, total: 31 µs
Wall time: 33.1 µs


torch.Size([4, 2880])

In [35]:
diff_mse(z_reshaped, torch_z_reshaped)

0.0

### relu

In [36]:
with torch.no_grad():
    %time z_relu = relu_vector(z_reshaped, device)
z_relu.shape

CPU times: user 306 µs, sys: 22 µs, total: 328 µs
Wall time: 242 µs


torch.Size([4, 2880])

In [37]:
with torch.no_grad():
    %time torch_z_relu = torch.relu(z_reshaped)
torch_z_relu.shape

CPU times: user 111 µs, sys: 8 µs, total: 119 µs
Wall time: 75.1 µs


torch.Size([4, 2880])

In [38]:
diff_mse(z_relu, torch_z_relu)

0.0

### fc

In [39]:
fc = torch.nn.Linear(z_reshaped.size(-1), N_classes)
w_fc = fc.weight
b_fc = fc.bias

In [40]:
with torch.no_grad():
    %time z_fc = fc_layer_vector(z_reshaped, w_fc, b_fc, device)
z_fc.shape

CPU times: user 331 µs, sys: 24 µs, total: 355 µs
Wall time: 188 µs


torch.Size([4, 10])

In [41]:
with torch.no_grad():
    %time torch_z_fc = fc(z_reshaped)
torch_z_fc.shape

CPU times: user 189 µs, sys: 14 µs, total: 203 µs
Wall time: 179 µs


torch.Size([4, 10])

In [42]:
diff_mse(z_fc, torch_z_fc)

1.4093761092331806e-15

## test

scalar model 10 * 1 batch - too expensive

with no_grad (no backward) ~300 times slower than vector model

In [43]:
# %time !python ./simple_conv_net_train.py --log-interval=1 --epochs 1 --type scalar --batch-size 1 --batch-limit 10

vector model 10 * 64 batch

In [44]:
%time !python ./simple_conv_net_train.py --log-interval=1 --epochs 1 --type vector --batch-limit 10

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.334315
Train Epoch: 1 [64/60000 (0%)]	Loss: 2.270116
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.251385
Train Epoch: 1 [192/60000 (0%)]	Loss: 2.199347
Train Epoch: 1 [256/60000 (0%)]	Loss: 2.146911
Train Epoch: 1 [320/60000 (1%)]	Loss: 2.005801
Train Epoch: 1 [384/60000 (1%)]	Loss: 2.041959
Train Epoch: 1 [448/60000 (1%)]	Loss: 2.009930
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.909573
Train Epoch: 1 [576/60000 (1%)]	Loss: 1.879718
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.715644

Test set: Average loss: -0.1930, Accuracy: 6252/10000 (62.52%)

CPU times: user 4.98 s, sys: 829 ms, total: 5.81 s
Wall time: 9min 52s


built-in model 10 * 64 batch

In [45]:
%time !python ./simple_conv_net_train.py --log-interval=1 --epochs 1 --batch-limit 10

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.334315
Train Epoch: 1 [64/60000 (0%)]	Loss: 2.270116
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.251385
Train Epoch: 1 [192/60000 (0%)]	Loss: 2.199347
Train Epoch: 1 [256/60000 (0%)]	Loss: 2.146911
Train Epoch: 1 [320/60000 (1%)]	Loss: 2.005801
Train Epoch: 1 [384/60000 (1%)]	Loss: 2.041959
Train Epoch: 1 [448/60000 (1%)]	Loss: 2.009930
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.909573
Train Epoch: 1 [576/60000 (1%)]	Loss: 1.879718
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.715644

Test set: Average loss: -0.1930, Accuracy: 6252/10000 (62.52%)

CPU times: user 17.5 ms, sys: 12.7 ms, total: 30.2 ms
Wall time: 2.22 s


## train

full built-in model train

In [46]:
%time !python ./simple_conv_net_train.py --log-interval=100 --epochs 20 --save-model

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.334315
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.319489
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.274875
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.362373
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.191254
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.173175
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.156882
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.124490
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.096554
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.095492

Test set: Average loss: -0.9235, Accuracy: 9592/10000 (95.92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.120768
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.056507
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.258464
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.117799
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.111027
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.037806
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.120253
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.069743
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.0332

Train Epoch: 15 [44800/60000 (75%)]	Loss: 0.002306
Train Epoch: 15 [51200/60000 (85%)]	Loss: 0.012122
Train Epoch: 15 [57600/60000 (96%)]	Loss: 0.002167

Test set: Average loss: -0.9841, Accuracy: 9881/10000 (98.81%)

Train Epoch: 16 [0/60000 (0%)]	Loss: 0.005876
Train Epoch: 16 [6400/60000 (11%)]	Loss: 0.002767
Train Epoch: 16 [12800/60000 (21%)]	Loss: 0.008380
Train Epoch: 16 [19200/60000 (32%)]	Loss: 0.003906
Train Epoch: 16 [25600/60000 (43%)]	Loss: 0.004492
Train Epoch: 16 [32000/60000 (53%)]	Loss: 0.032964
Train Epoch: 16 [38400/60000 (64%)]	Loss: 0.001586
Train Epoch: 16 [44800/60000 (75%)]	Loss: 0.016801
Train Epoch: 16 [51200/60000 (85%)]	Loss: 0.003574
Train Epoch: 16 [57600/60000 (96%)]	Loss: 0.007453

Test set: Average loss: -0.9839, Accuracy: 9878/10000 (98.78%)

Train Epoch: 17 [0/60000 (0%)]	Loss: 0.001103
Train Epoch: 17 [6400/60000 (11%)]	Loss: 0.002716
Train Epoch: 17 [12800/60000 (21%)]	Loss: 0.003643
Train Epoch: 17 [19200/60000 (32%)]	Loss: 0.032085
Train Epoch: 17

full vector model train

~30 hours to train! let's compare tiny piece (from test)

In [ ]:
%time !python ./simple_conv_net_train.py --log-interval=100 --epochs 20 --type vector --save-model

## summary

After `1` epoch of just `10` batches with size `64` (640 pics only) we did test on vector model and built-in pytorch

|model|time|accuracy|
|-|-|-|
|vector|9min 52s|62.52%|
|pytorch|2.22s|62.52%|

Python implementation even in vector form is almost 300 times slower than built-in pytorch

The accuracy is near the same value - models seem to work in a similar way

Scalar model is much slower than vector one - no results for comparison; but MSEs shows good quality of implementation

$$t_{scalar} \approx 300 \times t_{vector} \approx 10K \times t_{torch}$$
$$t_{vector} \approx 300 \times t_{torch}$$

All tests were at `Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, 4 Cores, 8 Threads` machine